## Working with feature geometries

To read the features from a feature collection (e.g a gpml or shapefile) from a file into memory, you need to use the following lines:

In [10]:
import pygplates

input_feature_filename = "Data/LIPs_2014.gpmlz"

features = pygplates.FeatureCollection(input_feature_filename)


The 'features' object contains all the features in that file - for example, every isochron in an isochron file. 

A simple way to to access individual features is to iterate over the features in a for loop. 

Each feature has properties (the ones you would see if you clicked on that feature in gplates) and a geometry (the point/line/polygon that gplates displays). We can access these for each feature, here is an example where for each feature within a feature collection, we print the Plate ID.

In [11]:
for feature in features:
    print feature.get_reconstruction_plate_id()

802
101
714
714
973
201
501
909
802
802
801
801
801
802
690
901
293
909
909
901
983
702
714
714
714
901
901
982
802
911
293
293
901
901
901
801
911
911
911
911
911
901
714
201
101
701
701
701
701
701
701
802
701
701
701
511
511
511
801
901
501
301
802
802
802
901
801
201
201
701
701
301
704
704
704
501
501
701
201
201
201
201
911
301
101
802
750
701
111
111
111
111


We can iterate over the features and look at the geometry type for each one

In [12]:
for feature in features:
    print feature.get_geometry()

This code combines both property and geometry capabilities.

It says, 'find me the features where the plate ID is 901, and show me the area of each polygon that have this plate ID'.

(NB the area is expressed for a unit sphere - to convert to Earth units, need to multiply by radius**2

In [13]:
for feature in features:
    if feature.get_reconstruction_plate_id() == 901:
        print feature.get_geometry().get_area()

0.00846239080491
0.00292433678503
0.000537599895546
0.00549003963242
7.47005726387e-05
5.42520011244e-05
0.000235709125096
0.00881938878497
0.0464521835569
0.0126459680057


Since we are often interested in having the point coordinates as latitudes and longitudes, there are a number of methods to give these to us.

For example, the following shows how to iterate over the polygons in a feature collection and, for polygons with plateid=101,

    - get the centroid point of the polygon
    - get the latitude and longitude of each centroid point

In [14]:
for feature in features:
    if feature.get_reconstruction_plate_id() == 101:
        polygon = feature.get_geometry()
        point = polygon.get_boundary_centroid()
        print point.to_lat_lon()


(32.32261854767031, -64.79063765137695)
(34.40311102132128, -76.23081963225276)
(37.12233943627294, -59.74171706937214)


To access the individual vertices of polygons, you can use this code:

In [15]:
for feature in features:
    if feature.get_reconstruction_plate_id() == 101:
        polygon = feature.get_geometry()
        points = polygon.get_points()
        print 'Polygon vertices are:'
        print points.to_lat_lon_list()

Polygon vertices are:
[(31.983469009399414, -64.52339935302734), (31.86470222473144, -64.643798828125), (31.87250328063965, -64.81924819946289), (31.88030433654785, -64.99469947814941), (31.888105392456055, -65.1701488494873), (31.89590263366699, -65.34560012817383), (32.06707191467286, -65.32503318786621), (32.238237380981445, -65.3044662475586), (32.40940284729004, -65.28389930725098), (32.521100997924805, -65.15929985046387), (32.632802963256836, -65.03469848632812), (32.7445011138916, -64.91009902954102), (32.85619926452637, -64.7854995727539), (32.80920219421387, -64.60643196105957), (32.762201309204094, -64.42736434936523), (32.715200424194336, -64.24829864501953), (32.55047035217285, -64.25973129272461), (32.3857364654541, -64.27116584777832), (32.22100257873536, -64.28260040283203), (32.10223579406738, -64.40299987792969), (31.983469009399414, -64.52339935302734)]
Polygon vertices are:
[(30.22346687316895, -79.63259887695312), (30.09639930725098, -79.7677993774414), (30.1265506

The following example shows how to reconstruct individual feature geometries using a rotation pole derived from a rotation file. This gives more fine-grained control on reconstructions than the basic 'pygplates.reconstruct' function, which is suitable for reconstructing all the features in a collection for a single reconstruction time. 

Here, we iterate over a series of feature geometries, determine a 'rotation' object based on the attributes of that polygon, and apply this rotation to the centroid point geometry using:

             rotation * polygon.get_boundary_centroid()

In [17]:
# Load a rotation file
input_rotation_filename = 'Data/Seton_etal_ESR2012_2012.1.rot'

rotation_model=pygplates.RotationModel(input_rotation_filename)

# Get finite rotation for plate 701 relative to spin axis at 52 Ma
rotation = rotation_model.get_rotation(52.,701, 0., 1)
print rotation

# For each LIP polygon feature with plateid 701, get the centroid location and reconstruct it 
# based on the rotation pole we just got from the rotation file
for feature in features:
    if feature.get_reconstruction_plate_id() == 701:
        polygon = feature.get_geometry()
        reconstructed_point = rotation * polygon.get_boundary_centroid()
        print 'Centroid Location = ',polygon.get_boundary_centroid().to_lat_lon()
        print 'Reconstructed Centroid Location = ',reconstructed_point.to_lat_lon()
        

(rot = (pole = (lat: 34.104, lon: -54.5802) (which is antipodal to (lat: -34.104, lon: 125.42)); angle = -10.6035 deg))
Centroid Location =  (-28.50417029230606, 3.494617659504184)
Reconstructed Centroid Location =  (-35.56272441776961, -5.738561928124286)
Centroid Location =  (-27.276294756047797, 3.6047190829327467)
Reconstructed Centroid Location =  (-34.35216756965727, -5.461119312412153)
Centroid Location =  (-33.751047439553076, 1.6697809256194587)
Reconstructed Centroid Location =  (-40.59844461766543, -8.46819111807814)
Centroid Location =  (-32.695875181514154, -0.1933082444225856)
Reconstructed Centroid Location =  (-39.36151877211106, -10.31956596273306)
Centroid Location =  (-31.964478946978502, 2.142512912499003)
Reconstructed Centroid Location =  (-38.87121484022802, -7.681219651789854)
Centroid Location =  (-31.169714187484765, -0.41068067254202967)
Reconstructed Centroid Location =  (-37.82425345891745, -10.315473964508488)
Centroid Location =  (-40.454691636264144, 25.